In [1]:
#!/usr/bin/env python
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt

In [2]:
import numpy as np, os, sys
from tqdm.notebook import tqdm    
from manipulations import get_classes, get_classes_from_header, get_Fs_from_header, load_challenge_data

if __name__ == '__main__':
    
    Datas = []
    Header_datas = []
    Classes = []
    for dataset in range(1,7):
        print('Dataset ', dataset)
        # Parse arguments.
        if len(sys.argv) != 3:
            raise Exception('Include the input and output directories as arguments, e.g., python driver.py input output.')

        input_directory = '../Data/Training{}/'.format(dataset)
        output_directory = '../Output/'

        # Find files.
        input_files = []
        for f in os.listdir(input_directory):
            if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
                input_files.append(f)

        if not os.path.isdir(output_directory):
            os.mkdir(output_directory)

        classes=get_classes(input_directory,input_files)

        num_files = len(input_files)
        datas = []
        header_datas = []
        for i, f in tqdm(enumerate(input_files)):
            #print('    {}/{}...'.format(i+1, num_files), f)
            tmp_input_file = os.path.join(input_directory,f)
            data,header_data = load_challenge_data(tmp_input_file)
            datas.append(data)
            header_datas.append(header_data)

        Datas += datas
        Header_datas += header_datas
        Classes += classes
        print('Done.')

Dataset  1



Done.
Dataset  2



Done.
Dataset  3



Done.
Dataset  4



Done.
Dataset  5



Done.
Dataset  6



Done.


In [4]:
from manipulations import get_abbr, get_name
from global_vars import labels, Dx_map, Dx_map_unscored
first_idx = {scored_code: None for scored_code in list(Dx_map['SNOMED CT Code'])}
first_idx_unscored = {unscored_code: None for unscored_code in list(Dx_map_unscored['SNOMED CT Code'])}

# for i, Header_data in tqdm(enumerate(Header_datas)):
#     codes = get_classes_from_header(Header_data)
#     abbrs = ' '.join([get_abbr(int(code), Dx_map, Dx_map_unscored) for code in codes])
#     for code, abbr in zip(codes, abbrs):
#         if code in first_idx and first_idx[code] is None:
#             first_idx[code] = i
#         if code in first_idx_unscored and first_idx_unscored[code] is None:
#             first_idx_unscored[code] = i
#     encore = False
#     for code in first_idx.keys():
#         if first_idx[code] is None:
#             encore = True
#             break
# #     for code in first_idx_unscored.keys():
# #         if first_idx_unscored[code] is None:
# #             encore = True
# #             break
#     if not encore:
#         break

In [5]:
from signal_processing import myfilter, main_QRST
Codes = []
Q_locs = []
for idx in tqdm(range(0, 11186)): 
    if np.sum(Datas[idx]) != 0: # not all zeros
        codes = get_classes_from_header(Header_datas[idx])
        names = ', '.join([get_name(int(code), Dx_map, Dx_map_unscored) for code in codes])

        filtered_Data = myfilter(Datas[idx][:,1000:4000], 500, vis=False)

        # get the lead to apply Pan Tomkins
        Q_loc = main_QRST(filtered_Data, idx, '', '', names, fig2=False)

        # store
        Codes.append(codes)
        Q_locs.append(Q_loc)
    else:
        Codes.append([])
        Q_locs.append([])

In [6]:
from global_vars import labels
def get_scored_class(code, labels):
    return [1 if label in code else 0 for label in labels]

In [7]:
data_labels = np.array([get_scored_class(codes, labels) for codes in Codes])
    

In [8]:
MAX_RR = 750 # 60 beats/min => 60 beats/60 s ==> beat/1s ==> 500 samples / beat

In [9]:
X_trains = []
Data_labels = []

for i in tqdm(range(100)):
    Q_loc = Q_locs[i]
    RR_avg = np.median([Q_loc[k+1] - Q_loc[k] for k in range(len(Q_loc)-1)])
    RR_th = (0.3 * RR_avg, 3 * RR_avg)
    
    ks = [k for k in range(len(Q_loc)-1) if Q_loc[k+1] - Q_loc[k] > RR_th[0] 
              and Q_loc[k+1] - Q_loc[k] < RR_th[1]]
        
    for k in ks:
        Data_labels.append(get_scored_class(Codes[i], labels))
        X_trains.append(Datas[i][:,1000+Q_loc[k]:1000+Q_loc[k+1]])

In [10]:
Signals = np.zeros((len(X_trains),12,MAX_RR))
for i in range(len(X_trains)):
    Signals[i,:,:min(len(X_trains[i][0]),MAX_RR)] = X_trains[i][:,:MAX_RR]

# On y va

In [60]:
from torch import nn
from torchvision import models

In [66]:
def activation_func(activation):
    return  nn.ModuleDict([
        ['relu', nn.ReLU(inplace=True)],
        ['leaky_relu', nn.LeakyReLU(negative_slope=0.01, inplace=True)],
        ['selu', nn.SELU(inplace=True)],
        ['none', nn.Identity()]
    ])[activation]

In [249]:
from functools import partial
class Conv1dAuto(nn.Conv1d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding =  (self.kernel_size[0] // 2,) # dynamic add padding based on the kernel_size
       # print(self.kernel_size, self.padding)
        
conv3 = partial(Conv1dAuto, kernel_size=3, bias=False)    

In [250]:
conv3(64,32)

Conv1dAuto(64, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)

In [251]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, activation='relu'):
        super().__init__()
        self.in_channels, self.out_channels, self.activation = in_channels, out_channels, activation
        self.blocks = nn.Identity()
        self.activate = activation_func(activation)
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        x = self.activate(x)
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels

In [252]:
ResidualBlock(32, 64)

ResidualBlock(
  (blocks): Identity()
  (activate): ReLU(inplace=True)
  (shortcut): Identity()
)

In [253]:
dummy = torch.ones((1, 1, 1))

block = ResidualBlock(1, 64)
block(dummy)

tensor([[[2.]]])

In [254]:
class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(
            nn.Conv1d(self.in_channels, self.expanded_channels, kernel_size=1,
                      stride=self.downsampling, bias=False),
            nn.BatchNorm1d(self.expanded_channels)) if self.should_apply_shortcut else None
        
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels

In [255]:
block = ResNetResidualBlock(32, 64)

In [256]:
def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(conv(in_channels, out_channels, *args, **kwargs), nn.BatchNorm1d(out_channels))


In [257]:
class ResNetBasicBlock(ResNetResidualBlock):
    """
    Basic ResNet block composed by two layers of 3conv/batchnorm/activation
    """
    expansion = 1
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, conv=self.conv, bias=False, stride=self.downsampling),
            activation_func(self.activation),
            conv_bn(self.out_channels, self.expanded_channels, conv=self.conv, bias=False),
        )

In [258]:
dummy = torch.ones((1, 32, 224))# in channel 1, out channel 32, length 224 (in 2d there is another dim)

block = ResNetBasicBlock(32, 64)
block(dummy).shape


torch.Size([1, 64, 224])

In [259]:
class ResNetBottleNeckBlock(ResNetResidualBlock):
    expansion = 4
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, expansion=4, *args, **kwargs)
        self.blocks = nn.Sequential(
           conv_bn(self.in_channels, self.out_channels, self.conv, kernel_size=1),
             activation_func(self.activation),
             conv_bn(self.out_channels, self.out_channels, self.conv, kernel_size=3, stride=self.downsampling),
             activation_func(self.activation),
             conv_bn(self.out_channels, self.expanded_channels, self.conv, kernel_size=1),
        )

In [260]:
dummy = torch.ones((1, 32, 10))

block = ResNetBottleNeckBlock(32, 64)
block(dummy).shape
print(block)

ResNetBottleNeckBlock(
  (blocks): Sequential(
    (0): Sequential(
      (0): Conv1dAuto(32, 64, kernel_size=(1,), stride=(1,), bias=False)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU(inplace=True)
    (2): Sequential(
      (0): Conv1dAuto(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): ReLU(inplace=True)
    (4): Sequential(
      (0): Conv1dAuto(64, 256, kernel_size=(1,), stride=(1,), bias=False)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (activate): ReLU(inplace=True)
  (shortcut): Sequential(
    (0): Conv1d(32, 256, kernel_size=(1,), stride=(1,), bias=False)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


In [272]:
class ResNetLayer(nn.Module):
    """
    A ResNet layer composed by `n` blocks stacked one after the other
    """
    def __init__(self, in_channels, out_channels, block=ResNetBasicBlock, n=1, *args, **kwargs):
        super().__init__()
        # 'We perform downsampling directly by convolutional layers that have a stride of 2.'
        downsampling = 2 if in_channels != out_channels else 1
        self.blocks = nn.Sequential(
            block(in_channels , out_channels, *args, **kwargs, downsampling=downsampling),
            *[block(out_channels * block.expansion, out_channels, downsampling=1, *args, **kwargs) for _ in range(n - 1)]
        )

    def forward(self, x):
        x = self.blocks(x)
        return x

In [273]:

dummy = torch.ones((1, 64, 48))

layer = ResNetLayer(64, 128, block=ResNetBasicBlock, n=3)
layer(dummy).shape

torch.Size([1, 128, 24])

In [275]:
class ResNetEncoder(nn.Module):
    """
    ResNet encoder composed by layers with increasing features.
    """
    def __init__(self, in_channels=3, blocks_sizes=[64, 128, 256, 512], deepths=[2,2,2,2], 
                 activation='relu', block=ResNetBasicBlock, *args, **kwargs):
        super().__init__()
        self.blocks_sizes = blocks_sizes
        
        self.gate = nn.Sequential(
            nn.Conv1d(in_channels, self.blocks_sizes[0], kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm1d(self.blocks_sizes[0]),
            activation_func(activation),
            nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        )
        
        self.in_out_block_sizes = list(zip(blocks_sizes, blocks_sizes[1:]))
        self.blocks = nn.ModuleList([ 
            ResNetLayer(blocks_sizes[0], blocks_sizes[0], n=deepths[0], activation=activation, 
                        block=block,*args, **kwargs),
            *[ResNetLayer(in_channels * block.expansion, 
                          out_channels, n=n, activation=activation, 
                          block=block, *args, **kwargs) 
              for (in_channels, out_channels), n in zip(self.in_out_block_sizes, deepths[1:])]       
        ])
        
        
    def forward(self, x):
        x = self.gate(x)
        for block in self.blocks:
            x = block(x)
        return x

class ResnetDecoder(nn.Module):
    """
    This class represents the tail of ResNet. It performs a global pooling and maps the output to the
    correct class by using a fully connected layer.
    """
    def __init__(self, in_features, n_classes):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool1d((1,))
        self.decoder = nn.Linear(in_features, n_classes)

    def forward(self, x):
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        return x
    
class ResNet(nn.Module):
    
    def __init__(self, in_channels, n_classes, *args, **kwargs):
        super().__init__()
        self.encoder = ResNetEncoder(in_channels, *args, **kwargs)
        self.decoder = ResnetDecoder(self.encoder.blocks[-1].blocks[-1].expanded_channels, n_classes)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [279]:
def resnet18(in_channels, n_classes, block=ResNetBasicBlock, *args, **kwargs):
    return ResNet(in_channels, n_classes, block=block, deepths=[2, 2, 2, 2], *args, **kwargs)

resnet = resnet18(64, len(labels))

In [280]:
len(labels)

27

In [281]:
resnet(dummy)

tensor([[-0.3010, -0.4552, -0.8846, -0.9471,  0.0468,  0.0338,  0.4211, -0.6411,
          0.1713, -0.8693,  0.2292,  0.1981,  0.1014, -0.1080,  0.6074, -0.6452,
         -0.2831,  0.2506,  0.9028, -0.5637,  0.4700, -0.7781,  1.5727, -0.7719,
         -0.0652, -0.3378,  0.8847]], grad_fn=<AddmmBackward>)

In [168]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/resnet1d')


# Phew.. let's do some toy examples

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [24]:
from resnet1d import resnet18
import torch
from torch import nn

In [13]:
from global_vars import weights

In [16]:

def weighted_binary_cross_entropy2(sigmoid_x, y, weighted_matrix, weight=None, reduction=None):
    """
    Aha this is correct!
    sigmoid_x = nn.Sigmoid()(x)
    Args:
        sigmoid_x: predicted probability of size [N,C], N sample and C Class. Eg. Must be in range of [0,1]
        targets: true value, one-hot-like vector of size [N,C]
        pos_weight: Weight for postive sample
    """
    if not (y.size() == sigmoid_x.size()):
        raise ValueError("Target size ({}) must be the same as input size ({})".format(y.size(), sigmoid_x.size()))
   
    #print("y.size(), sigmoid_x.size()", y.size(), sigmoid_x.size())
    sigmoid_x = torch.clamp(sigmoid_x,min=1e-7,max=1-1e-7) 
    loss = - torch.matmul(y*sigmoid_x.log() + (1-y)*(1-sigmoid_x).log(), weighted_matrix)
    
    if weight is not None:
        loss = loss * weight
        
    if reduction is None:
        return loss
    elif reduction == 'mean':
        return loss.mean()
    elif reduction == 'sum':
        return loss.sum()
    return None
    
class WeightedBCELoss(nn.Module):
    def __init__(self, weights, PosWeightIsDynamic= False, WeightIsDynamic= False, 
                 reduction='mean'):
        """
        Args:
            pos_weight = Weight for postive samples. Size [1,C]
            weight = Weight for Each class. Size [1,C]
            PosWeightIsDynamic: If True, the pos_weight is computed on each batch. If pos_weight is None, then it remains None.
            WeightIsDynamic: If True, the weight is computed on each batch. If weight is None, then it remains None.
        """
        super().__init__()

        self.register_buffer('weights', weights)
        self.reduction = reduction
        self.PosWeightIsDynamic = PosWeightIsDynamic

    def forward(self, input, target):
        if self.PosWeightIsDynamic:
            positive_counts = target.sum(dim=0)
            nBatch = len(target)
            self.pos_weight = (nBatch - positive_counts)/(positive_counts +1e-5)


        return weighted_binary_cross_entropy2(input, target,
                                             weighted_matrix=self.weights,
                                             reduction=self.reduction)

In [18]:
Signal_12leads = np.transpose(Signals, (1,0,2))

In [19]:
from torch.utils.data import Dataset
from torchvision import transforms
from global_vars import labels
import os
class SignalDataset(Dataset):

    def __init__(self, signals, labels):
        self.signals = signals
        self.labels = labels

    def __len__(self):
        return len(self.signals)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample =(torch.Tensor(np.array([self.signals[idx]]).transpose()), torch.Tensor(self.labels[idx]))

        return sample
    
signal_datasets = SignalDataset(Signal_12leads[0,:,:], Data_labels)

In [22]:
signal_datasets[0][1].size()

torch.Size([27])

In [27]:
model = resnet18(MAX_RR, len(labels))
model.to(device)

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv1d(750, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBasicBlock(
            (blocks): Sequential(
              (0): Sequential(
                (0): Conv1dAuto(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
                (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): ReLU(inplace=True)
              (2): Sequential(
                (0): Conv1dAuto(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
                (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [28]:
weights = torch.Tensor(weights).to(device)

In [46]:
from torchvision import datasets, models, transforms
from myeval import agg_y_preds_bags, binary_acc, geometry_loss
import torch.optim as optim
from torch.optim import lr_scheduler
from snippets.pytorchtools import EarlyStopping
from sklearn.model_selection import KFold
import time

st = time.time()
patience = 50
kf = KFold(5)
batch_size=10000

saved_dir = '../saved/ResNet1d/'
X = Signal_12leads[0,:,:]
y = Data_labels

for i, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    
    trainDataset = torch.utils.data.Subset(signal_datasets, train_idx)
    testDataset = torch.utils.data.Subset(signal_datasets, test_idx)
    
    trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size, shuffle = True, pin_memory=True)#sampler = sampler)
    trainLoader2 = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size, shuffle = False, pin_memory=True)#sampler = sampler)
    testLoader = torch.utils.data.DataLoader(testDataset, batch_size = batch_size, shuffle = False, pin_memory=True)

    
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) #
    # Decay LR by a factor of 0.1 every 100 epochs
    scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    criterion_train = WeightedBCELoss(weights=weights, reduction='mean')
    criterion_test = WeightedBCELoss(weights=weights, reduction='mean')

    losses_train = []
    losses_test = []

    avg_losses_train = []
    avg_losses_test = []


    early_stopping = EarlyStopping(patience, verbose=False, 
                                  saved_dir=saved_dir, 
                                   save_name='MutliCWTNetFull11'+str(i))
    epoch = 0
    auroc = 0
    auprc = 0
    accuracy = 0
    fmeasure = 0
    fbeta = 0
    gbeta = 0
    for epoch in range(50000):
        
        model.train()
        output_trains = []
        
        for X_train, y_train in trainLoader:
            y_train = y_train.to(device)
            X_train = X_train.to(device)
            optimizer.zero_grad()
            output_train = model(X_train)
            loss_train = criterion_train(output_train, y_train)
            #print(loss_train)
            losses_train.append(loss_train.item())
            loss_train.backward()
            optimizer.step()

        scheduler.step()
        
        avg_loss_train = np.average(losses_train)
        avg_losses_train.append(avg_loss_train)
        
        output_tests = []
        y_tests = []
        y_trains = []
        with torch.no_grad():
            model.eval()

            for X_train, y_train in trainLoader2:  
                X_train = X_train.to(device)
                output_train = model(X_train)
                output_trains.append(output_train.cpu())
                y_trains.append(y_train.cpu())
    
                output_trains.append(output_train.cpu())
                y_trains.append(y_train)
                
            for X_test, y_test in testLoader:  
                y_test = y_test.to(device)
                X_test = X_test.to(device)
                output_test = model(X_test)
                
                loss_test = criterion_test(output_test, y_test)
                losses_test.append(loss_test.item())
                
                output_tests.append(output_test.cpu())
                y_tests.append(y_test.cpu())
                
            avg_loss_test = np.average(losses_test)
            avg_losses_test.append(avg_loss_test)
        
        y_trains = torch.cat(y_trains, axis=0)
        y_tests = torch.cat(y_tests, axis=0)
    
        output_trains = torch.cat(output_trains, axis=0)
        y_train_preds = torch.sigmoid(output_trains)
        
        output_tests = torch.cat(output_tests, axis=0)
        y_test_preds = torch.sigmoid(output_tests)
        
        #output_trains = torch.cat(output_trains, axis=0)
#         y_train_preds_max, y_train_preds_mean, _ = agg_y_preds_bags(y_train_preds, bag_size=n_segments)
#         y_test_preds_max, y_test_preds_mean, _ = agg_y_preds_bags(y_test_preds, bag_size=n_segments)
#         _, _, y_trains = agg_y_preds_bags(y_trains, bag_size=n_segments)
#         _, _, y_tests = agg_y_preds_bags(y_tests, bag_size=n_segments)
        



        if epoch % 500 == 0:
            
            acc, fmeasure, fbeta, gbeta = binary_acc(y_train_preds, y_trains)           
            acc2, fmeasure2, fbeta2, gbeta2 = binary_acc(y_test_preds, y_tests)
            geometry = geometry_loss(fbeta, gbeta)
            geometry2 = geometry_loss(fbeta2, gbeta2)
            output_str = 'S{}/{} {:.2f} min |\n Train Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f} |\n Valid Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f}\n '.format(
                i, epoch, (time.time()-st)/60,
                avg_loss_train, acc, fmeasure, fbeta, gbeta, geometry,
                avg_loss_test, acc2, fmeasure2, fbeta2, gbeta2, geometry2)
            print(output_str)
        
#             output_str = 'S{}/{} {:.2f} min Train Loss: {:.6f} Valid Loss: {:.6f}'.format(i, epoch, (time.time()-st)/60, avg_loss_train, avg_loss_test)
#             print(output_str)
#             with open(saved_dir+'loss11_{}.txt'.format(i), 'a') as f:
#                 print(output_str, file=f)

#             early_stopping(avg_loss_test, model)

            if early_stopping.early_stop:
                print("Early stopping")
                break
            
#     output_string = 'AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure|Geomotry\n{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}'.format(auroc2,auprc2,acc2,fmeasure2,fbeta2,gbeta2,geometry2)
#     print(output_string)     
#     with open(saved_dir+'score'+ str(i)+ '_epoch' + str(epoch) + '.txt', 'w') as f:
#         f.write(output_string)

#     avg_losses_train = np.array(avg_losses_train)
#     avg_losses_test = np.array(avg_losses_test)
    
#     np.save(saved_dir + 'avg_losses_train' + str(i) + '_epoch' + str(epoch), avg_losses_train)
#     np.save(saved_dir + 'avg_losses_test' + str(i) + '_epoch' + str(epoch), avg_losses_test)
    